In [1]:
# Import necessary module for SQL
from sqlalchemy import create_engine

In [2]:
# Create a Engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
# Connect the Engine
con = engine.connect()

In [4]:
# Execute the commands to see the tables
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [5]:
# Checking the structure of each table
import pandas as pd
df_bookings = pd.DataFrame(con.execute('SELECT * FROM Bookings').fetchall())
df_bookings.columns = con.execute('SELECT * FROM Bookings').keys()

In [6]:
df_bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [7]:
df_facilities = pd.DataFrame(con.execute('SELECT * FROM Facilities').fetchall())
df_facilities.columns = con.execute('SELECT * FROM Facilities').keys()
df_facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [8]:
df_members = pd.DataFrame(con.execute('SELECT * FROM Members').fetchall())
df_members.columns = con.execute('SELECT * FROM Members').keys()
df_members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [9]:
# Solution to Q10
df_ans10 = pd.read_sql_query("""SELECT f.name AS 'Facility_Name',
                                       SUM(CASE WHEN memid <> 0 THEN f.membercost*b.slots
                                           ELSE f.guestcost*b.slots END) AS 'Revenue'
                                FROM Bookings AS b
                                LEFT Join Facilities AS f
                                USING(facid)
                                GROUP BY f.name
                                HAVING Revenue < 1000
                                ORDER BY Revenue""", engine)

In [10]:
print(df_ans10)

   Facility_Name  Revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [11]:
# Solution to Q11
df_ans11 = pd.read_sql_query("""SELECT (m1.surname || ', ' || m1.firstname) AS 'Member_Name',
                                       (m2.surname || ', ' || m2.firstname) AS 'Recommended_By'
                                FROM Members AS m1
                                LEFT JOIN Members AS m2
                                ON m1.recommendedby = m2.memid
                                WHERE (m1.recommendedby <> '')
                                ORDER BY Member_Name""", engine)

In [12]:
print(df_ans11)

                 Member_Name      Recommended_By
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

In [13]:
# Solution to Q12
# The duration of slot is half an hour so usage can be calculated by multiplying the number of slots with 30 minutes
# and summing them up
df_ans12 = pd.read_sql_query("""SELECT f.name AS 'Facility_Name',
                                       SUM(b.slots*30) AS 'Usage_in_mins'
                                FROM Bookings AS b
                                LEFT JOIN Facilities as f
                                USING(facid)
                                WHERE b.memid <> 0
                                GROUP BY f.name
                                ORDER BY Usage_in_mins""", engine)

In [14]:
print(df_ans12)

     Facility_Name  Usage_in_mins
0   Massage Room 2           1620
1     Squash Court          12540
2     Table Tennis          23820
3       Pool Table          25680
4    Snooker Table          25800
5   Tennis Court 2          26460
6   Massage Room 1          26520
7   Tennis Court 1          28710
8  Badminton Court          32580


In [15]:
# Solution to Q13
df_ans13 = pd.read_sql_query("""SELECT f.name AS 'Facility_Name',
                                       SUM(b.slots*30) AS 'Usage_in_mins',
                                       strftime('%m', b.starttime) as 'Month'
                                FROM Bookings AS b
                                LEFT JOIN Facilities as f
                                USING(facid)
                                WHERE b.memid <> 0
                                GROUP BY strftime('%m', b.starttime), f.name
                                ORDER BY f.name, Month""", engine)

In [16]:
print(df_ans13)

      Facility_Name  Usage_in_mins Month
0   Badminton Court           4950    07
1   Badminton Court          12420    08
2   Badminton Court          15210    09
3    Massage Room 1           4980    07
4    Massage Room 1           9480    08
5    Massage Room 1          12060    09
6    Massage Room 2            240    07
7    Massage Room 2            540    08
8    Massage Room 2            840    09
9        Pool Table           3300    07
10       Pool Table           9090    08
11       Pool Table          13290    09
12    Snooker Table           4200    07
13    Snooker Table           9480    08
14    Snooker Table          12120    09
15     Squash Court           1500    07
16     Squash Court           5520    08
17     Squash Court           5520    09
18     Table Tennis           2940    07
19     Table Tennis           8880    08
20     Table Tennis          12000    09
21   Tennis Court 1           6030    07
22   Tennis Court 1          10170    08
23   Tennis Cour

In [17]:
con.close()